# Exercise 1: Tool Calling

Large Language Models (LLMs) are incredibly powerful. However, as we've seen, they can struggle with simple mathematical problems and are limited by the information contained in their training data.

That said, we can describe tools, such as Python functions, to the LLM that may be useful for solving tasks. While the models themselves won't call the functions directly, they can determine when a function is needed and return the appropriate arguments for use.

In this notebook, you'll learn how describing functions to a model can transform it into an agent capable of reasoning with itself to answer a query.

In [ ]:
# autoreload imports
%load_ext autoreload

%autoreload 2

In [ ]:
from llm_in_production.llm import instantiate_langchain_model
from llm_in_production.agent_utils import (
    # function_to_json,
    tool_calling_agent,
)
from langchain_core.utils.function_calling import convert_to_openai_function
import dotenv
import numpy as np
import os
import json
import requests
import pandas as pd
import yfinance as yf
import tiktoken

# This reads the .env file in your project and transforms its content into env variables.
# This way you don't have to hard code your secrets.
dotenv.load_dotenv()

# Here we create the client. 
# Make sure you select the LLM provider that corresponds to the one you are using in this course!
client = instantiate_langchain_model(
    # llm_provider="azure",
    llm_provider="gcp",
)
client.model_name

## LLM limiations

Let's start by demonstrating one of the limitations of LLMs.

The prompt below is asking the model to answer a simple highschool mathematics problem.

In [ ]:
math_problem = "what is the circumference of a circle with radius 5.31cm"

In [ ]:
system_prompt = "You are a calculator bot that is used to answer mathematics problems"

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": math_problem},
]


In [ ]:

response = client.invoke(
    input=messages,
    seed=0,
)

message = response.content

print(message)

However, the answer it produces is incorrect.

$2*\pi*5.31 = 33.3637139811...$

For this problem, a simple Python function would be more useful!

In [ ]:
def circumference_calculator(radius: float, something: float = 4.4) -> float:
    """Calculates the circumference of a circle given the radius

    :param radius: The radius of the circle
    :return: The circumference of the circle
    """
    return 2 * np.pi * radius


circumference_calculator(5.31)

But what if the LLM knew about this Python function?

## Tool Calling

It is now possible to inform LLMs about external tools.

The model can then determine on its own when it's appropriate to use one of these tools and, if needed, return the appropriate arguments.

To describe a function to the model it must be in a particular JSON format. LangChain have created a helpful function to transform Python functions into this format.

In [ ]:
circumference_calculator_json = convert_to_openai_function(circumference_calculator)
circumference_calculator_json

Let's give the model access to this tool and repeat the prompt we did before.

In [ ]:
tools = [{"type": "function", "function": circumference_calculator_json}]

response = client.invoke(
    input=messages,
    seed=0,
    tools=tools,
    tool_choice = "auto"  # (default setting) the model will pick between generating a message or calling a function automatically 
)
response

The model has no message to return:

In [ ]:
message = response.content
print(f"Message: {message}")

But, it has called for a tool to be used, with arguments!

In [ ]:
tool_calls = response.tool_calls
print(f"Tool calls: {tool_calls}")

Let's do what the model suggests and call the Python function `circumference_calculator`, with the argument `radius=5.31`.

In [ ]:
tool_name = tool_calls[0]['name']
tool_args = tool_calls[0]['args']
tool_response = json.dumps(eval(tool_name)(**tool_args))

print(f"Function response {tool_response}")

The number is correct. Let's pass information back to the model.

First we need to update the message history.

In [ ]:
messages.append(response)

messages.append(
    {
        "role": "tool",
        "name": tool_name,
        "content": tool_response,
        "tool_call_id": tool_calls[0]['id'],
    }
)  # extend conversation with function response

if len(messages) > 4:
    raise Exception(
        "Too many messages have been added! Restart and rerun the notebook."
    )

messages

Now we can call the model again to answer the original query.

In [ ]:
response = client.invoke(
    input=messages,
    seed=0,
    tools=tools,
)  # get a new response using the result of the function call


message = response.content
tool_calls = response.tool_calls

print(f"New tool calls: {len(tool_calls)}")
print(message)

Thanks to tool calling, we were able to produce the correct result!

Notice how the model will only reach for a function if it is needed.

In [ ]:
history_problem = "How many wives did king Henry VIII of England have?"

response = client.invoke(
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": history_problem},
    ],
    seed=0,
    tools=tools,
)

message = response.content
tool_calls = response.tool_calls

print(f"Message: {message}")
print(f"Tool calls: {tool_calls}")

## External APIs

One way of gathering live information is through the use of an external API.

They can enable us to retrieve a wide variety of information (although some will come at a cost for commercial usage). For example:
- The weather conditions for a particular location.
- The topic headline from a particular news source.
- The recent prices of a stock .


### Weather

To get information about local weather we can use the API from https://www.weatherapi.com, which is availble on a [free plan](https://www.weatherapi.com/pricing.aspx). An API key has been provided for you in the Codespace.

We can use the function below to obtain weather information for a particular location. You can take a look at the [WeatherAPI docs](https://www.weatherapi.com/docs/) for more examples of how to use the API.

In [ ]:
def get_current_weather(location: str) -> dict:
    """Get the current weather conditions in a given location

    :param location: The city (and state), e.g. "San Francisco, CA"
    :return: The weather conditions
    """
    return requests.get(
        f"https://api.weatherapi.com/v1/current.json?key={os.environ['WEATHER_API_KEY']}&q={location}"
    ).json()


get_current_weather("London")

There is a lot of information in the JSON that the function returns, more than is needed for the average person. However, an LLM would be able to summarize this information into a readable paragraph.

In the cell below, we have written a system prompt and user prompt, which we pass to the `tool_calling_agent` function, along with the `get_current_weather` function.

The `tool_calling_agent` is a helpful function we wrote to automate the reasoning process used above:

- First, any functions or tools we want to give the model access to are transformed into the JSON format it expects.
- We then extract a response from the model based on the input prompts.
- The process may finish there, but if the model deems it necessary, it may ask for a tool to be called. In this case:
    - The tool is called using the parameters returned by the model.
    - The tool’s response is then given to the model.
    - Finally, the output based on this new information is returned.
    
We’ve added some logging to help you understand what’s going on "under the hood."


In [ ]:
system_prompt = """
You are a weather bot that is able to give back a summary of the weather conditions in the given location. 

You do not return many measurements about the conditions.

Instead, you give an overall idea as to what conditions are like in one or two sentences."
"""

user_prompt = "What is the weather in London?"

In [ ]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_current_weather)
print(output)

### Exercise 1a: News

Your task is to build an agent that can summarize the top headlines of the day from a given news source.

To give the model access to the latest news stories, you can use the https://newsapi.org/ API, which is available on a [free plan](https://newsapi.org/pricing). An API key is already provided for you in the Codespace.


#### Part i: The day's top headlines

Write a Python function that returns JSON information about the top headlines of the day (from a given source) in the Python dictionary format. You can use the examples given in the [NewsAPI docs](https://newsapi.org/docs/endpoints/top-headlines) to help you answer this question.

In [ ]:
def get_top_headlines(source: str) -> dict:
    # YOUR CODE HERE START
    # YOUR CODE HERE END

get_top_headlines("bbc-news")

#### Part ii: Prompt engineering for a news summary

Write a system prompt and user prompt that will lead the agent to summarize the day's top headlines. 

Rather than simply returning a list of the top stories, the agent should be able to condense the information into a few sentences. The agent should also be capable of only writing about the news category the user is interested in, e.g. politics or sport.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

In [ ]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_top_headlines)
print(output)

### Exercise 1b: Stocks

In this exercise you will build a stock analysis agent to help a user to make investment decisions.

The agent will be able examine the recent history of a stock, along with recent press articles about it, to determine whether or not the stock is a good investment.

<mark>Warning! This agent is only intended to be used for educational purpose and we do not expect its suggestions to be of much worth.</mark> 

<mark> Xebia will not be held liable for any financial losses incurred from its usage!</mark> 

To get the recent history of a stock we will use [yfinance](https://github.com/ranaroussi/yfinance). yfinance offers a threaded and Pythonic way to download market data from [Yahoo!Ⓡ finance](https://finance.yahoo.com). Note that it is **not** affiliated, endorsed, or vetted by Yahoo, Inc. It's an open-source tool that uses Yahoo's publicly available APIs, and is intended for research and educational purposes.

Given a stock ticker symbol, we can retrive recent information about that stock.

In [ ]:
stock = yf.Ticker("IBM")
df = stock.history(period="1mo")
df = df[["Close", "Volume"]]
print(df.index.max() - df.index.min())
df.index = [str(x).split()[0] for x in list(df.index)]
df.index.rename("Date", inplace=True)
df

#### Part i: Recent stock history

Write a Python function that returns JSON information about the recent history of a particular stock in the Python dictionary format.

In [ ]:
def get_stock_prices(company_stock_ticker_symbol: str, period: str = "1mo") -> dict:
    # YOUR CODE HERE START
    # YOUR CODE HERE END
    return df.to_json()

In [ ]:
get_stock_prices("IBM")

We can use NewsAPI again to retrieve the recent headlines about a particular stock.

In [ ]:
today = pd.Timestamp.today()
start = pd.Timestamp.today()
end = today - pd.Timedelta(days=30)

In [ ]:
topic = "IBM stock news"

request = requests.get(
    f"https://newsapi.org/v2/everything?q={topic}&from={start}&to={end}&sortBy=popularity&apiKey={os.environ['NEWS_API_KEY']}"
).json()
articles = pd.DataFrame(request["articles"])
titles = articles["title"]
titles


#### Part ii: Any recent news?

Write a Python function that returns JSON information about the recent headlines of a particulaer stock in the Python dictionary format.

In [ ]:
def get_news_stories(topic: str) -> dict:
    # YOUR CODE HERE START
    # YOUR CODE HERE END
    return titles.to_json()

In [ ]:
get_news_stories("Apple")

#### Part iii: Prompt engineering for a stock

Write a system prompt and user prompt that will lead the agent to determine whether or not a stock is a good investment. 

As you may have already experienced with ChatGPT, the model will want to caveat its opinions.
For the purpose of this assignment, see if you can get it to provide an opinion without any reservations.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

In [ ]:
output = tool_calling_agent(client, system_prompt, user_prompt, get_news_stories, get_stock_prices)
print(output)

#### Part iv: Prompt engineering for multiple stocks.

Update your system/user prompt so that it will lead the agent to make an investment decision for more than one stock.

In [ ]:
# YOUR CODE HERE START
# YOUR CODE HERE END

---